In [1]:
# Generate a large prime p

import random
import math

def is_prime(n):
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    for i in range(3, math.isqrt(n) + 1, 2):
        if n % i == 0:
            return False
    return True

def generate_prime(bits):
    while True:
        p = random.getrandbits(bits)
        p |= (1 << bits - 1) | 1
        if is_prime(p):
            return p

p = generate_prime(8)
print(p)

179


In [2]:
# Define the Galois field GF(p)

class GF:
    def __init__(self, p):
        self.p = p

    def add(self, a, b):
        return (a + b) % self.p

    def sub(self, a, b):
        return (a - b) % self.p

    def mul(self, a, b):
        return (a * b) % self.p

    def div(self, a, b):
        return (a * pow(b, self.p - 2, self.p)) % self.p

    def pow(self, a, b):
        return pow(a, b, self.p)

    def inv(self, a):
        return pow(a, self.p - 2, self.p)
    
    def det(self, a):
        return (a[0][0] * a[1][1] - a[0][1] * a[1][0]) % self.p

    def sqrt(self, a):
        return pow(a, (self.p + 1) // 4, self.p)

    def rand(self):
        return random.randint(0, self.p - 1)
    
    def __str__(self):
        return f"GF({self.p})"
    
    def __repr__(self):
        return str(self)
    
gf = GF(p)
print(gf)

GF(179)


In [3]:
# Choose an irreducible Selmer cubic polynomial f(x) over Q[x]

def is_irreducible(f):
    for i in range(1, len(f) // 2 + 1):
        if len(f) % i != 0:
            continue
        for j in range(1, len(f) // i):
            if f[:i] * j == f:
                return False
    return True

def generate_irreducible(p, n):
    while True:
        f = [gf.rand() for _ in range(n)]
        f[-1] = 1
        if is_irreducible(f):
            return f

f = generate_irreducible(p, 3)
print(f)

[139, 48, 1]


In [4]:
# Define an elliptic curve E over GF(p) using f(x)

class EllipticCurve:
    def __init__(self, gf, f):
        self.gf = gf
        self.f = f

    def add(self, p, q):
        x1, y1 = p
        x2, y2 = q
        if p == q:
            m = self.gf.div(self.gf.add(self.gf.mul(3, self.gf.mul(x1, x1)), self.f[1]), self.gf.mul(2, y1))
        else:
            m = self.gf.div(self.gf.sub(y2, y1), self.gf.sub(x2, x1))
        x3 = self.gf.sub(self.gf.sub(self.gf.mul(m, m), x1), x2)
        y3 = self.gf.sub(self.gf.mul(m, self.gf.sub(x1, x3)), y1)
        return x3, y3
    
    def sub(self, p, q):
        return self.add(p, (q[0], self.gf.sub(0, q[1])))

    def mul(self, p, n):
        q = (0, 1)
        for _ in range(n.bit_length()):
            q = self.add(q, q)
            if n & 1:
                q = self.add(q, p)
            n >>= 1
        return q

    def rand(self):
        x = self.gf.rand()
        y = self.gf.sqrt(self.gf.add(self.gf.add(self.gf.mul(x, x), self.gf.mul(x, self.f[0])), self.f[1]))
        return x, y

    def __str__(self):
        return f"EllipticCurve({self.gf}, {self.f})"
    
    def __repr__(self):
        return str(self)
    
ec = EllipticCurve(gf, f)
print(ec)

EllipticCurve(GF(179), [139, 48, 1])


In [5]:
# Select a random integer d in [1, p - 1] as the private key

d = random.randint(1, p - 1)

# compute the public key P = d * G where G is the base point of E

G = ec.rand()
P = ec.mul(G, d)

# Compute Q = f(P) using the Selmer cubic polynomial f(x)

Q = ec.mul(P, 3)

# Generate a random nxn invertible matrix B over GF(p)

def generate_invertible(p, n):
    while True:
        B = [[gf.rand() for _ in range(n)] for _ in range(n)]
        if math.isclose(gf.pow(gf.det(B), gf.inv(n)), 1):
            return B

B = generate_invertible(p, 3)

# Return (P,Q,B) as the public key and d as the private key

print(f'Public key (P,Q,B): ({P}, {Q}, {B})', f'Private key d: {d}', sep='\n')


Public key (P,Q,B): ((160, 169), (177, 5), [[62, 149, 137], [94, 44, 48], [32, 5, 119]])
Private key d: 176


In [6]:
# Encryption Algorithm

# Plaintext message M is a simple string
M = 'Hello, world!'

# Plaintext message M into a point M_p on the elliptic curve E
M_p = ec.rand()

# Generate a random integer k in [1, p - 1]
k = random.randint(1, p - 1)

# Compute the ciphertext C = (k * G, M_p + k * Q)
C = ec.add(ec.mul(G, k), ec.add(M_p, ec.mul(Q, k)))

# Return C as the ciphertext
print(f'Ciphertext C: {C}')

Ciphertext C: (129, 121)


In [7]:
# Decryption Algorithm

# Compute shared secret S = d * C[0]
S = ec.mul(C[0], d)

# Recover the transformed message point M_p' = C[1] - S
M_p_prime = ec.sub(C[1], S)

# Compute the inverse matrix B_inv of B over GF(p)
B_inv = [[gf.inv(B[j][i]) for j in range(3)] for i in range(3)]

# Recover the original message point M_p = B_inv * M_p'
M_p = [sum([gf.mul(B_inv[i][j], M_p_prime[j]) for j in range(3)]) % p for i in range(3)]

# Return M_p as the plaintext message
print(f'Plaintext message M_p: {M_p}')

TypeError: cannot unpack non-iterable int object